# Confidence interval

In [1]:
import os
# Scientific libraries
import numpy as np
import scipy
import pandas as pd

# Graphic libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='ticks', context='talk')

# Creating alias for magic commands
%alias_magic t time

#tools for modeling
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV

Created `%t` as an alias for `%time`.
Created `%%t` as an alias for `%%time`.


In [2]:
!pip install statsmodels

In [3]:
import statsmodels.formula.api as smf

In [4]:
# generate data following a line with some noise
df = pd.DataFrame({'x': np.linspace(0, 1, 10)})
a, b = 10, 1
df['y'] = a * df['x'] + b + np.random.rand(*df['x'].shape)

In [5]:
results = smf.ols('y ~ x', data=df).fit()

In [6]:
print(results.summary2())

                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.991   
Dependent Variable: y                AIC:                7.7968  
Date:               2020-07-10 15:17 BIC:                8.4020  
No. Observations:   10               Log-Likelihood:     -1.8984 
Df Model:           1                F-statistic:        948.7   
Df Residuals:       8                Prob (F-statistic): 1.34e-09
R-squared:          0.992            Scale:              0.10699 
-------------------------------------------------------------------
            Coef.    Std.Err.      t      P>|t|    [0.025    0.975]
-------------------------------------------------------------------
Intercept   1.5766     0.1922    8.2007   0.0000   1.1332    2.0199
x           9.9826     0.3241   30.8010   0.0000   9.2352   10.7300
-----------------------------------------------------------------
Omnibus:              3.317        Durbin-Watson:           2.708
Prob(Omnibus):   

C:\Users\Victor\Anaconda3\envs\updated\lib\site-packages\scipy\stats\stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))


In [7]:
from scipy import stats

In [8]:
?stats.sem

In [9]:
# ex 7.3e in "prob & stats for engineers & scientists"
# 95% CI
sample = [5, 8.5, 12, 15, 7, 9, 7.5, 6.5, 10.5]

df = len(sample) - 1
m = np.mean(sample)
se = stats.sem(sample)
stats.t.interval(0.95, df, m, se)

(6.630805969849321, 11.369194030150679)

Now let us see what happen if we increase sample size.

In [10]:
def compute_CI(sample):
    df = len(sample) - 1
    m = np.mean(sample)
    se = stats.sem(sample)
    return stats.t.interval(0.95, df, m, se)

In [11]:
sample0 = [5, 8.5, 12, 15, 7, 9, 7.5, 6.5, 10.5]
for i in range(1, 11):
    sample = sample0 * i
    print(compute_CI(sample))

(6.630805969849321, 11.369194030150679)
(7.513018465798657, 10.486981534201343)
(7.828553205228008, 10.171446794771992)
(8.002826688375313, 9.997173311624687)
(8.117096031193578, 9.882903968806422)
(8.19938533664971, 9.80061466335029)
(8.26227193997882, 9.73772806002118)
(8.312347246626748, 9.687652753373252)
(8.353442036736979, 9.646557963263021)
(8.387954443106528, 9.612045556893472)


We can see that CIs get smaller and smaller when the sample size increases, which means larger sample sizes give better estimation of the true mean of population.

## CI for difference of means of 2 populations

In [12]:
# parse data in example
rows = [
    '36 54 52 60',
    '44 52 64 44',
    '41 37 38 48',
    '53 51 68 46',
    '38 44 66 70',
    '36 35 52 62'
]
aa, bb = [], []
for r in rows:
    values = r.split()
    aa += [float(s) for s in values[:2]]
    bb += [float(s) for s in values[2:]]

aa += [34, 44]
print(aa)
print(bb)

[36.0, 54.0, 44.0, 52.0, 41.0, 37.0, 53.0, 51.0, 38.0, 44.0, 36.0, 35.0, 34, 44]
[52.0, 60.0, 64.0, 44.0, 38.0, 48.0, 68.0, 46.0, 66.0, 70.0, 52.0, 62.0]


In [13]:
?stats.norm

In [14]:
ma, mb = np.mean(aa), np.mean(bb)
var_a = 40; var_b = 100
se = np.sqrt(var_a/len(aa) + var_b/len(bb))
rv = stats.norm(loc=ma - mb, scale=se)
rv.interval(0.95)

(-19.604123716241833, -6.491114378996268)

So we can say with a prob 95% that the diff between two means will fall in the interval from $-19.6$ to $-6.5$.

If the variances of the populations are __unknown but equal__, then we can use std errors of the samples as estimate for variances.

In [15]:
a_se, b_se = stats.sem(aa), stats.sem(bb)
se2 = np.sqrt(a_se**2/len(aa) + b_se**2/len(bb))
rv2 = stats.norm(loc=ma - mb, scale=se2)
rv2.interval(0.95)

(-15.042009165534358, -11.053228929703744)

## CI for proportions (mean of Bernoulli distribution)

Ex 7.5a:

In a sample of 100 transistors, 80 of them meet standards. 
What are 95%, 98% CIs for the proportion of transistors meeting standards in all transistors.

There are two ways, easier is to use module `proportion` in statsmodels. The other is to compute z-critical value by `scipy` then use formula. Below I will do both, for the sake of practice.

### Via `statsmodels.stats.proportion`

In [26]:
from statsmodels.stats.proportion import proportion_confint, proportion_effectsize

In [18]:
# 95% CI
proportion_confint(80, 100, alpha=0.05)

(0.7216014406183978, 0.8783985593816023)

In [19]:
# 98% CI
proportion_confint(80, 100, alpha=0.02)

(0.7069460850383664, 0.8930539149616337)

So we can say with a confidence level 95% that the true portion of transistors meeting standards is somewhere between 72% and 87.8%.

### Via computing z-critical value

z-critical value can be computed given confidence level via method `ppf` (inverse of `cdf`). Actually, given a prob value $p_{cum}$, the funciton `ppf` returns the value $x$ such that the $cdf(x) = p_{cum}$. Thus, the first thing we should do is to convert conf. level to prob value $p_{cum}$, then we can use `ppf`.

__NOTE:__ as CI is a two-tailed estimation, the prob value $p$ for a given conf. lvl $c$ will be: $ p_{cum} = c + (1-c)/2 $. In other words, the $z$-value is  $z_{\alpha / 2}$

In [23]:
def lookup_z_score(alpha):
    c = 1 - alpha # conf. lvl.
    p_cum = c + (1-c)/2
    z_value = stats.norm.ppf(p_cum)
    return z_value

def compute_CI(c, count, n):
    p_cum = c + (1-c)/2
    z_value = stats.norm.ppf(p_cum)
    phat = count/n
    se = np.sqrt(phat*(1 - phat)/n)
    return phat - z_value*se, phat + z_value*se

In [21]:
compute_CI(0.95, 80, 100)

(0.7216014406183979, 0.8783985593816022)

In [22]:
compute_CI(0.98, 80, 100)

(0.7069460850383664, 0.8930539149616337)

## CI, error and sample size

This part shows relationship between CI/error and sample size via examples. Though the examples are limited to proportion estimations, the same principle applies to other cases.

The examples also illustrate the rule of thumb that error and sample size have an inverse relationship.

__EXAMPLE 7.5b:__ On October 14, 2003, the New York Times reported that a recent poll
indicated that $52$ percent of the population was in favor of the job performance of
President Bush, with a margin of error of $±4$ percent. What does this mean? Can we
infer how many people were questioned?

For proportion estimations, the error is: $ \epsilon = z_{\alpha/2}  \sqrt{\hat{p}(1- \hat{p})/n} $. Thus, given $\hat{p}$ and $\alpha$ aka $1- $ conf. lvl. , we can compute sample size for each desired error $\epsilon$ as:
$$ n \approx \hat{p}(1- \hat{p}) * z^2_{\alpha/2} * \epsilon^{-2} $$

It is a common practice that media report a CI of $95\%$ or $\alpha=5\%$.

In [24]:
alpha = 0.05
z_score = lookup_z_score(alpha)
phat = 0.52
err = 0.04
n = phat*(1 - phat)* z_score**2 * err**(-2)
print(n)

599.2675760282835


Similarly, we can estimate a sample size for which we can obtain a CI whose length not exceeds a given bound $b$ via formula:
$$ n \approx \hat{p}(1- \hat{p}) * (2z_{\alpha/2})^2 * b^{-2} $$

That is, if $k$ items were initially sampled to obtain the preliminary estimate $\hat{p} $, then an
additional $n − k$ (or 0 if $n \leq k$) items should be sampled.

__EXAMPLE 7.5c:__ A certain manufacturer produces computer chips; each chip is independently
acceptable with some unknown probability p. To obtain an approximate 99 percent
confidence interval for p, whose length is approximately .05, an initial sample of 30
chips has been taken. If 26 of these chips are of acceptable quality, then the preliminary
estimate $\hat{p}$ is 26/30. 

Using this $ \hat{p} $, a 99 percent CI of length $b=.05$ will require a sample size:

In [30]:
phat = 26/30
b = .05
z_score = lookup_z_score(alpha=0.01)
n = phat*(1 - phat) * (2*z_score)**2 * b**(-2)
print(round(n))

1227.0


In [29]:
1227 - 30

1197

As the sample size requires about 1227 chips, we need to sample 1197 more chips.

In [31]:
# test if this sample size really helps to achieve a CI with desired length
n_obs = round(n)
count = phat * n_obs
lci, uci = compute_CI(0.99, count, n_obs)
print(uci - lci)

0.04999426740681545


__REMARK__

If we know true $p$ then $ n= p(1-p) * (2z_{\alpha/2})^2 * b^{-2} $. 

As $ p(1-p) \leq 1/4, \forall p $, an upper bound for $n$ is $ z^2_{\alpha/2} * b^{-2} $. Thus, by choosing a sample whose size is at least as large as this upper bound, one can be assured of obtaining a confidence interval of length no greater than $b$ without need of any additional sampling.